# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [27]:
# Write your code below.
%reload_ext dotenv
%dotenv /Users/hadiahussain/Desktop/Assignments/production/01_materials/labs/.env

In [28]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [29]:
import os
from glob import glob

# Write your code below.
PRICE_DATA = os.getenv("PRICE_DATA")
parquet_files = glob(os.path.join(PRICE_DATA, "**/*.parquet"), recursive=True)


In [35]:
parquet_files

['../../05_src/data/prices/AAPL/AAPL_2010/part.0.parquet',
 '../../05_src/data/prices/AAPL/AAPL_2010/part.1.parquet',
 '../../05_src/data/prices/AAPL/AAPL_2017/part.0.parquet',
 '../../05_src/data/prices/AAPL/AAPL_2017/part.1.parquet',
 '../../05_src/data/prices/AAPL/AAPL_2021/part.0.parquet',
 '../../05_src/data/prices/AAPL/AAPL_2021/part.1.parquet',
 '../../05_src/data/prices/AAPL/AAPL_2019/part.0.parquet',
 '../../05_src/data/prices/AAPL/AAPL_2019/part.1.parquet',
 '../../05_src/data/prices/AAPL/AAPL_2018/part.0.parquet',
 '../../05_src/data/prices/AAPL/AAPL_2018/part.1.parquet',
 '../../05_src/data/prices/AAPL/AAPL_2020/part.0.parquet',
 '../../05_src/data/prices/AAPL/AAPL_2020/part.1.parquet',
 '../../05_src/data/prices/AAPL/AAPL_2016/part.0.parquet',
 '../../05_src/data/prices/AAPL/AAPL_2016/part.1.parquet',
 '../../05_src/data/prices/AAPL/AAPL_2011/part.0.parquet',
 '../../05_src/data/prices/AAPL/AAPL_2011/part.1.parquet',
 '../../05_src/data/prices/AAPL/AAPL_2002/part.0.parquet

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [38]:
# Write your code below.
import dask.dataframe as dd
import numpy as np

dd_px = dd.read_parquet(parquet_files).set_index("Ticker")
# Add lags for Close and Adj_Close variables and calculate returns
dd_feat = (dd_px.groupby('Ticker', group_keys=False).apply(
    lambda x: x.assign(Close_lag_1 = x['Close'].shift(1))
).assign(
    returns = lambda x: x['Close']/x['Close_lag_1'] - 1
).assign(
    hi_lo_range=lambda x: x['High'] - x['Low']
))

dd_feat

/var/folders/k1/k29czyyx30342rxl7m25j79h0000gn/T/ipykernel_41149/3458132343.py:7: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_feat = (dd_px.groupby('Ticker', group_keys=False).apply(


,Date,Adj Close,Close,High,Low,Open,Volume,Year,Close_lag_1,returns,hi_lo_range
npartitions=64,,,,,,,,,,,
AAPL,"datetime64[ns, UTC]",float64,float64,float64,float64,float64,float64,int32,float64,float64,float64
ACN,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...
ZBRA,...,...,...,...,...,...,...,...,...,...,...
ZBRA,...,...,...,...,...,...,...,...,...,...,...


+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [40]:
# Write your code below.
pd_feat = dd_feat.compute()

pd_feat['rolling_avg_returns'] = (
    pd_feat.groupby('Ticker')['returns']  # Group by ticker
    .transform(lambda x: x.rolling(window=10).mean())  # Apply rolling mean
)

pd_feat.head(15)


,Date,Adj Close,Close,High,Low,Open,Volume,Year,Close_lag_1,returns,hi_lo_range,rolling_avg_returns
Ticker,,,,,,,,,,,,
AAPL,2000-01-03 00:00:00+00:00,0.843077,0.999442,1.004464,0.907924,0.936384,5.357968e+08,2000,NaN,NaN,0.096540,NaN
AAPL,2000-01-04 00:00:00+00:00,0.771997,0.915179,0.987723,0.903460,0.966518,5.123776e+08,2000,0.999442,-0.084310,0.084263,NaN
AAPL,2000-01-05 00:00:00+00:00,0.783293,0.928571,0.987165,0.919643,0.926339,7.783216e+08,2000,0.915179,0.014633,0.067522,NaN
AAPL,2000-01-06 00:00:00+00:00,0.715508,0.848214,0.955357,0.848214,0.947545,7.679728e+08,2000,0.928571,-0.086538,0.107143,NaN
AAPL,2000-01-07 00:00:00+00:00,0.749402,0.888393,0.901786,0.852679,0.861607,4.607344e+08,2000,0.848214,0.047369,0.049107,NaN
AAPL,2000-01-10 00:00:00+00:00,0.736221,0.872768,0.912946,0.845982,0.910714,5.050640e+08,2000,0.888393,-0.017588,0.066964,NaN
AAPL,2000-01-11 00:00:00+00:00,0.698563,0.828125,0.887277,0.808036,0.856585,4.415488e+08,2000,0.872768,-0.051151,0.079241,NaN
AAPL,2000-01-12 00:00:00+00:00,0.656668,0.778460,0.852679,0.772321,0.848214,9.760688e+08,2000,0.828125,-0.059973,0.080358,NaN
AAPL,2000-01-13 00:00:00+00:00,0.728689,0.863839,0.881696,0.825893,0.843610,1.032685e+09,2000,0.778460,0.109677,0.055803,NaN


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?
It was not necessary to convert to pandas to calculate the moving average. 
Dask is usually used when the dataset is too large. if the dataset is small enough for the computers memory to handle then pandas is recommended. 
(1 pt)

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.